In [1]:
import urllib.request
import pandas as pd
import re
from io import StringIO
import numpy as np

# Input coordinates

In [2]:
inputCoords=[4289663.4482,  -4018945.9158,  -2467135.6259]
inputVelocity=[0,0,0]
inputEpoch=2000.4
inputItrf='ITRF2000'
outputEpoch=2020.62 #YEAR + DOY/365
outputItrf='ITRF2014' # This is only working for ITRF2014 for now

# Downloading ITRF data

In [3]:
url="https://itrf.ign.fr/doc_ITRF/Transfo-ITRF2014_ITRFs.txt"

In [4]:
file_name="parameters.txt"
#urllib.request.urlretrieve(url, file_name)
response = urllib.request.urlopen(url)
itrfFile = response.readlines()

In [5]:
l=[]
lastLine=''
for line in itrfFile:
  line=line.decode("utf-8").strip()
  if line.startswith("rates") or line.startswith("RATES"):
    l.append(lastLine+" "+line)
  elif line.startswith("ITRF") or line.startswith("SOLUTION"):
    lastLine=line
sio=StringIO('\n'.join(l))
itrfdf=pd.read_csv(sio,delim_whitespace=True)
itrfdf

,SOLUTION,Tx,Ty,Tz,D,Rx,Ry,Rz,EPOCH,RATES,Tx.1,Ty.1,Tz.1,D.1,Rx.1,Ry.1,Rz.1
0,ITRF2008,1.6,1.9,2.4,-0.02,0.00,0.00,0.00,2010.0,rates,0.0,0.0,-0.1,0.03,0.00,0.00,0.00
1,ITRF2005,2.6,1.0,-2.3,0.92,0.00,0.00,0.00,2010.0,rates,0.3,0.0,-0.1,0.03,0.00,0.00,0.00
2,ITRF2000,0.7,1.2,-26.1,2.12,0.00,0.00,0.00,2010.0,rates,0.1,0.1,-1.9,0.11,0.00,0.00,0.00
3,ITRF97,7.4,-0.5,-62.8,3.80,0.00,0.00,0.26,2010.0,rates,0.1,-0.5,-3.3,0.12,0.00,0.00,0.02
4,ITRF96,7.4,-0.5,-62.8,3.80,0.00,0.00,0.26,2010.0,rates,0.1,-0.5,-3.3,0.12,0.00,0.00,0.02
5,ITRF94,7.4,-0.5,-62.8,3.80,0.00,0.00,0.26,2010.0,rates,0.1,-0.5,-3.3,0.12,0.00,0.00,0.02
6,ITRF93,-50.4,3.3,-60.2,4.29,-2.81,-3.38,0.40,2010.0,rates,-2.8,-0.1,-2.5,0.12,-0.11,-0.19,0.07
7,ITRF92,15.4,1.5,-70.8,3.09,0.00,0.00,0.26,2010.0,rates,0.1,-0.5,-3.3,0.12,0.00,0.00,0.02
8,ITRF91,27.4,15.5,-76.8,4.49,0.00,0.00,0.26,2010.0,rates,0.1,-0.5,-3.3,0.12,0.00,0.00,0.02
9,ITRF90,25.4,11.5,-92.8,4.79,0.00,0.00,0.26,2010.0,rates,0.1,-0.5,-3.3,0.12,0.00,0.00,0.02


In [6]:
itrfData=itrfdf[itrfdf['SOLUTION']==inputItrf]
itrfData

,SOLUTION,Tx,Ty,Tz,D,Rx,Ry,Rz,EPOCH,RATES,Tx.1,Ty.1,Tz.1,D.1,Rx.1,Ry.1,Rz.1
2,ITRF2000,0.7,1.2,-26.1,2.12,0.0,0.0,0.0,2010.0,rates,0.1,0.1,-1.9,0.11,0.0,0.0,0.0


# Coordinate update

In [7]:
deltaEpoch=outputEpoch-inputEpoch
updatedCoords=np.array(inputCoords)+deltaEpoch*np.array(inputVelocity)

# ITRF Convertion

In [14]:
deltaRates=outputEpoch-itrfData['EPOCH'].values[0]
#=[itrfData['Tx']+itrfData['Tx.1']*deltaRates, itrfData['Ty']+itrfData['Ty.1']*deltaRates, itrfData['Tz']+itrfData['Tz.1']*deltaRates]
T=itrfData[['Tx','Ty','Tz']].to_numpy()[0]/1000. #meters per year
rateT=itrfData[['Tx.1','Ty.1','Tz.1']].to_numpy()[0]/1000.
T=T+ rateT*deltaRates
R=itrfData[['Rx','Ry','Rz']].to_numpy()[0]/1000. #arc seconds per year
rateR=itrfData[['Rx.1','Ry.1','Rz.1']].to_numpy()[0]/1000.
R=R*rateR*deltaRates/3600 #degrees per year
D=itrfData[['D']].to_numpy()[0]/1000. #arc seconds per year
rateD=itrfData[['D.1']].to_numpy()[0]/1000.
D=D*rateD*deltaRates/10e8 #degrees per year
print("T",T)
print("D",D)
R=np.array([[D[0], -R[2],  R[1]],
            [R[2],  D[0], -R[0]],
            [R[1],  R[0],  D[0]]  ])

print("R",R)

T [ 0.001762  0.002262 -0.046278]
D [2.476584e-15]
R [[ 2.476584e-15 -0.000000e+00  0.000000e+00]
 [ 0.000000e+00  2.476584e-15 -0.000000e+00]
 [ 0.000000e+00  0.000000e+00  2.476584e-15]]


# Convertion

In [15]:
#xs=x + T + R @ x caso fosse 2014 para 2000
x=updatedCoords
I=np.identity(3)
xs=np.linalg.inv(R+I) @ (x-T)
print ("coordenadas referenciadas ao ITRF 2014 época",outputEpoch,":",xs)

coordenadas referenciadas ao ITRF 2014 época 2020.62 : [ 4289663.44643799 -4018945.91806199 -2467135.57962199]


# Backwards convertion

In [16]:
T

array([ 0.001762,  0.002262, -0.046278])

In [17]:
inputCoords=[4289663.4482,  -4018945.9158,  -2467135.6259]
deltaEpoch=inputEpoch-outputEpoch
updatedCoords=np.array(inputCoords)+deltaEpoch*np.array(inputVelocity)
x=updatedCoords
I=np.identity(3)
xs=x + T + R @ x 
print ("coordenadas referenciadas ao ITRF 2014 época",outputEpoch,":",xs)

coordenadas referenciadas ao ITRF 2014 época 2020.62 : [ 4289663.44996201 -4018945.91353801 -2467135.67217801]
